<a href="https://colab.research.google.com/github/Gandhravs/Gandhrav_Sharma_AER850_Project3/blob/main/AER_850_PROJECT_3_FINAL_TRY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# starting on friday 12/13/2024

In [ ]:
# Step 1: Object Masking with Contour Extraction

# Import necessary libraries
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define the path to the input image
image_path = "/content/drive/MyDrive/AER_850_DATA/data/motherboard_image.JPEG"

# Load and preprocess the image
original_image = cv2.imread(image_path, cv2.IMREAD_COLOR)
rotated_image = cv2.rotate(original_image, cv2.ROTATE_90_CLOCKWISE)

# Apply Gaussian blur to smoothen the image
blurred_image = cv2.GaussianBlur(original_image, (47, 47), 4)

# Convert to grayscale for further processing
gray_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2GRAY)
rotated_gray_image = cv2.rotate(gray_image, cv2.ROTATE_90_CLOCKWISE)

# Perform adaptive thresholding to segment the image
threshold_image = cv2.adaptiveThreshold(
    rotated_gray_image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 55, 7)

# Detect edges using the Canny edge detector
edges_detected = cv2.Canny(threshold_image, 50, 350)
dilated_edges = cv2.dilate(edges_detected, None, iterations=12)

# Identify contours from the edges
detected_contours, _ = cv2.findContours(dilated_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Prepare a blank mask to draw the largest contour
contour_mask = np.zeros_like(rotated_image)

# Find and draw the largest contour
largest_contour = max(detected_contours, key=cv2.contourArea)
cv2.drawContours(
    image=contour_mask, contours=[largest_contour], contourIdx=-1, color=(255, 255, 255), thickness=cv2.FILLED
)

# Apply the mask to the original rotated image

final_masked_image = cv2.bitwise_and(contour_mask, rotated_image)

# change to rgb
edges_rgb = cv2.cvtColor(edges_detected, cv2.COLOR_BGR2RGB)
mask_rgb = cv2.cvtColor(contour_mask, cv2.COLOR_BGR2RGB)
final_masked_rgb = cv2.cvtColor(final_masked_image, cv2.COLOR_BGR2RGB)


# Display the result

plt.figure(figsize=(20, 5))
plt.subplot(141)
plt.imshow(edges_rgb)
plt.title('Edge Detection')
plt.axis('off')

plt.subplot(143)
plt.imshow(mask_rgb)
plt.title('Masked Image')
plt.axis('off')

plt.subplot(144)
plt.imshow(final_masked_rgb)
plt.title('Final Extracted PCB')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Step 2 Yolo V8 install

!pip install ultralytics

In [ ]:
# Step 2 Yolo V8

from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

model.train(data="/content/drive/MyDrive/Project_Data/data/data.yaml" , epochs=175, imgsz = 1200, batch = 3, name = 'yolo_model_project_rasp')  # train the model